# Reto Star Wars

Preguntas:
  - ¿Qué película es la favorita de los fans?
  - ¿Qué personaje es el menos querido por todos?
  - ¿Quién disparó primero? ¿Cuántas personas acertaron y cuántas fallaron con esta pregunta?
  - ¿Afecta la edad en elegir al personaje favorito?

Importar módulos necesarios

In [1]:
import pandas as pd
import numpy as np

Lectura de datos

In [2]:
star_wars = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/star-wars-survey/StarWars.csv", encoding="ISO-8859-1")
star_wars.head(3)

,RespondentID,Have you seen any of the 6 films in the Star Wars franchise?,Do you consider yourself to be a fan of the Star Wars film franchise?,Which of the following Star Wars films have you seen? Please select all that apply.,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,"Please state whether you view the following characters favorably, unfavorably, or are unfamiliar with him/her.",Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Which character shot first?,Are you familiar with the Expanded Universe?,Do you consider yourself to be a fan of the Expanded Universe?æ,Do you consider yourself to be a fan of the Star Trek franchise?,Gender,Age,Household Income,Education,Location (Census Region)
0,NaN,Response,Response,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Han Solo,Luke Skywalker,Princess Leia Organa,Anakin Skywalker,Obi Wan Kenobi,Emperor Palpatine,Darth Vader,Lando Calrissian,Boba Fett,C-3P0,R2 D2,Jar Jar Binks,Padme Amidala,Yoda,Response,Response,Response,Response,Response,Response,Response,Response,Response
1,3.292880e+09,Yes,Yes,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,3,2,1,4,5,6,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Unfamiliar (N/A),Unfamiliar (N/A),Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,I don't understand this question,Yes,No,No,Male,18-29,NaN,High school degree,South Atlantic
2,3.292880e+09,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Male,18-29,"$0 - $24,999",Bachelor degree,West South Central


In [3]:
star_wars.columns = ["ID", "SeenMovies", "IsFan", "Ep1Seen", "EpIISeen", "EpIIISeen", "EpSeenIV", "EpSeenV", "EpSeenVI", "RankI", "RankII", "RankIII", "RankIV", "RankV", "RankVI", "HanFav", "LukeFav", "LeilaFav", "AnakinFav", "ObiFav", "EmperorFav", "DarthFav", "LandoFav", "BobaFav", "C_3P0Fav", "R2D2Fav", "JarFav", "PadmeFav", "YodaFav", "ShotFirst", "FamiliarExpandedUniverse", "FanExpandedUniverse", "fanStarTrek", "Gender", "Age", "HouseIncome", "Education", "LocationCensusRegion"]
star_wars = star_wars.drop(columns="ID")
star_wars.head(5)

,SeenMovies,IsFan,Ep1Seen,EpIISeen,EpIIISeen,EpSeenIV,EpSeenV,EpSeenVI,RankI,RankII,RankIII,RankIV,RankV,RankVI,HanFav,LukeFav,LeilaFav,AnakinFav,ObiFav,EmperorFav,DarthFav,LandoFav,BobaFav,C_3P0Fav,R2D2Fav,JarFav,PadmeFav,YodaFav,ShotFirst,FamiliarExpandedUniverse,FanExpandedUniverse,fanStarTrek,Gender,Age,HouseIncome,Education,LocationCensusRegion
0,Response,Response,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Han Solo,Luke Skywalker,Princess Leia Organa,Anakin Skywalker,Obi Wan Kenobi,Emperor Palpatine,Darth Vader,Lando Calrissian,Boba Fett,C-3P0,R2 D2,Jar Jar Binks,Padme Amidala,Yoda,Response,Response,Response,Response,Response,Response,Response,Response,Response
1,Yes,Yes,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,3,2,1,4,5,6,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Unfamiliar (N/A),Unfamiliar (N/A),Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,I don't understand this question,Yes,No,No,Male,18-29,NaN,High school degree,South Atlantic
2,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Male,18-29,"$0 - $24,999",Bachelor degree,West South Central
3,Yes,No,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,NaN,NaN,NaN,1,2,3,4,5,6,Somewhat favorably,Somewhat favorably,Somewhat favorably,Somewhat favorably,Somewhat favorably,Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),Unfamiliar (N/A),I don't understand this question,No,NaN,No,Male,18-29,"$0 - $24,999",High school degree,West North Central
4,Yes,Yes,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,5,6,1,2,4,3,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,Somewhat favorably,Very favorably,Somewhat favorably,Somewhat unfavorably,Very favorably,Very favorably,Very favorably,Very favorably,Very favorably,I don't understand this question,No,NaN,Yes,Male,18-29,"$100,000 - $149,999",Some college or Associate degree,West North Central


Checar Na's

In [4]:
star_wars.isna().sum(axis=0)

SeenMovies                    0
IsFan                       350
Ep1Seen                     513
EpIISeen                    615
EpIIISeen                   636
EpSeenIV                    579
EpSeenV                     428
EpSeenVI                    448
RankI                       351
RankII                      350
RankIII                     351
RankIV                      350
RankV                       350
RankVI                      350
HanFav                      357
LukeFav                     355
LeilaFav                    355
AnakinFav                   363
ObiFav                      361
EmperorFav                  372
DarthFav                    360
LandoFav                    366
BobaFav                     374
C_3P0Fav                    359
R2D2Fav                     356
JarFav                      365
PadmeFav                    372
YodaFav                     360
ShotFirst                   358
FamiliarExpandedUniverse    358
FanExpandedUniverse         973
fanStarT

Seleccionar solo aquellos que vieron las películas de Star Wars

In [5]:
star_wars_seen = star_wars[star_wars['SeenMovies'] == "Yes"]

#### ¿Qué película es la favorita de los fans?
Seleccionar los datos (si son fans o no)

In [6]:
star_wars_no_fan_nas = star_wars_seen[star_wars_seen['IsFan'].notna()]
star_fans = star_wars_no_fan_nas[(star_wars_no_fan_nas["IsFan"] == "Yes")]
star_fans_rank = star_fans[["RankI", "RankII", "RankIII", "RankIV", "RankV", "RankVI"]]
star_fans_rank = star_fans_rank.dropna(axis=0)
star_fans_rank = star_fans_rank.astype("int")
star_fans_rank.sum(axis=0).idxmax() # El que tenga la menor suma es la película favorita

'RankIII'

#### ¿Qué personaje es el menos querido por todos?
Seleccionar los datos (si tienen datos para los favoritos)

In [7]:
star_favs = star_wars_seen[["HanFav", "LukeFav", "LeilaFav", "AnakinFav", "ObiFav", "EmperorFav", "DarthFav", "LandoFav", "BobaFav", "C_3P0Fav", "R2D2Fav", "JarFav", "PadmeFav", "YodaFav"]]
star_favs.isna().sum(axis=0)
star_favs = star_favs.dropna(axis=1, how="all")
(star_favs == "Very favorably").sum(axis=0) # más 'Very favorably' es el favorito
(star_favs == "Very unfavorably").sum(axis=0).idxmax() # más 'Very unfavorably' es el menos favorito

'JarFav'

#### ¿Quién disparó primero? ¿Cuántas personas acertaron y cuántas fallaron con esta pregunta?
Yo no entiendo la pregunta, entonces también eliminamos a los que no entendieron la pregunta

In [8]:
star_shot_first = star_wars_seen["ShotFirst"]
star_shot_first.unique() # observamos las posibles respuestas
star_shot_first = star_shot_first.dropna()
star_shot_first = star_shot_first[star_shot_first != "I don't understand this question"]
star_shot_first.value_counts() # La suma es el número de votos para cada una de las posibles respuestas

Han       325
Greedo    197
Name: ShotFirst, dtype: int64

#### ¿Afecta la edad en elegir al personaje favorito?
Podemos intentar responder la pregunta que ya se hizo antes, pero agruándo por edades

In [9]:
star_favs = star_wars_seen[["Age", "HanFav", "LukeFav", "LeilaFav", "AnakinFav", "ObiFav", "EmperorFav", "DarthFav", "LandoFav", "BobaFav", "C_3P0Fav", "R2D2Fav", "JarFav", "PadmeFav", "YodaFav"]]
star_favs.isna().sum(axis=0)
star_favs = star_favs.dropna()
star_favs.astype('category')
pd.concat([star_favs["Age"], (star_favs.iloc[:, 1:] == "Very favorably")], axis=1).groupby(by=["Age"]).sum().idxmax(axis=1) # más 'Very favorably' es el favorito y ahora agrupamos por edades

Age
18-29    YodaFav
30-44    YodaFav
45-60     HanFav
> 60      ObiFav
dtype: object